In [0]:
# Basic imports.
import os
import time
import random
import numpy as np
import torch

from torch import nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from sklearn import metrics

from torchtext import data
from torchtext import datasets

from matplotlib import pyplot as plt
%matplotlib inline

cudnn.benchmark = True

SEED = 1234
torch.manual_seed(SEED)

In [0]:
# Setting predefined arguments.
args = {
    'epoch_num': 10,     # Number of epochs.
    'lr': 3e-5,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 6,     # Number of workers on data loader.
    'batch_size': 10,     # Mini-batch size.
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


# Bidirectional RNN

Documentação Pytorch: https://pytorch.org/docs/stable/nn.html#torch.nn.RNN

Redes recorrentes bidirecionais, internamente implementam duas camadas recorrentes, cada qual recebendo a informação em uma ordem diferente. O objetivo é acumular conhecimento da sequência em ambas as direções, adquirindo contexto do passado (esquerda para direita) e do futuro (direita para a esquerda).

![](https://drive.google.com/uc?export=view&id=158b3Y_o-7yXsKMe5VdaifhPg0Du4A6dp)

Na implementação, o que muda em relação à camada unidirecional é:
*  Ao instanciar a camada recorrente, defina o parâmetro **```bidirectional = True```**
*  O hidden state tem dimensionalidade **```(num_layers * 2, batch_size, hidden_size)```**
*  O output tem dimensionalidade **```(seq_len, batch_size, hidden_size * 2)```**



In [0]:
class BRNN(nn.Module):
  
  def __init__(self,input_size, hidden_size, num_layers, batch_size):
    super(BRNN, self).__init__()
    
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    
    self.brnn = nn.RNN(input_size, hidden_size, num_layers, bidirectional=True)
    
        
  def forward(self, X):
    
    h0 = torch.zeros(self.num_layers * 2, self.batch_size, self.hidden_size).to(args['device'])
    
    output, hn = self.brnn(X, h0)
    print(output.size(), hn.size())
    return output
    

hidden_size   = 64
num_layers    = 1
input_size    = 1 
seq_len       = 140

net = BRNN(input_size,
             hidden_size, num_layers,
             args['batch_size']).to(args['device'])

# Generate random batch
X = torch.randn(seq_len, args['batch_size'], input_size).to(args['device'])
output = net(X)

torch.Size([140, 10, 128]) torch.Size([2, 10, 64])


# Long Short-Term Memory

A unidade recorrente básica produz apenas uma saída, o seu estado interno $h_t$. Este mesmo estado alimenta tanto as camadas subsequentes quanto a conexão recorrente da unidade. Por essa razão, ao realizar o forward em uma camada recorrente, precisamos inicializar o hidden state $h_t$, e alimentar a rede com o par de parâmetros $x_t, h_{t-1}$ a cada timestep.

```python
# Instanciando a camada
self.rnn = nn.RNN(input_size,  hidden_size, num_layers)

# Forward
h0 = torch.randn(num_layers, batch_size, hidden_size).to(args['device'])
output, hn = self.rnn(X, h0)
```

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" width="650">

A LSTM por outro lado produz duas saídas, o já conhecido hidden state $h_t$, mas também o cell state $c_t$. É graças ao cell state que a LSTM consegue mitigar o problema do vanishing gradient. De forma análoga aos blocos residuais e densos, que introduzem operações mais estáveis para preservar a força do gradiente, o cell state é atualizado através de uma soma.

Na prática, isso interfere na forma como fazemos o forward nessa rede, que agora precisa de dois estados iniciais ```(h0, c0)```, junto com a entrada, como apresentado no trecho de código a seguir.

```python
# Instanciando a camada
self.lstm = nn.LSTM(input_size,  hidden_size, num_layers)

# Forward
h0 = torch.randn(num_layers, batch_size, hidden_size).to(args['device'])
c0 = torch.randn(num_layers, batch_size, hidden_size).to(args['device'])

output, (hn, cn) = self.lstm(X, (h0, c0))
```

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" width="650">

In [0]:
class BiLSTM(nn.Module):
  
  def __init__(self,input_size, hidden_size, num_layers, batch_size):
    super(BiLSTM, self).__init__()
    
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    
    self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True)
    
        
  def forward(self, X):
    
    h0 = torch.zeros(self.num_layers * 2, self.batch_size, self.hidden_size).to(args['device'])
    c0 = torch.zeros(self.num_layers * 2, self.batch_size, self.hidden_size).to(args['device'])
    
    output, (hn, cn) = self.bilstm(X, (h0, c0)) 
    print(output.size(), hn.size(), cn.size())
    return output
    

hidden_size   = 64
num_layers    = 1
input_size    = 1 
seq_len       = 140

net = BiLSTM(input_size,
             hidden_size, num_layers,
             args['batch_size']).to(args['device'])

# Generate random batch
X = torch.randn(seq_len, args['batch_size'], input_size).to(args['device'])
output = net(X)

torch.Size([140, 10, 128]) torch.Size([2, 10, 64]) torch.Size([2, 10, 64])


# Empacotando Sequências

O pacote de funções de rnn, ```nn.utils.rnn```, oferece meios de processar batches contendo sequências de tamanho variável. Isso é realizado através do **padding** da sequência (ex: preenchimento com zeros),  de modo que elas aparentem ter igual comprimento, porém internamente as posições preenchidas não são processadas pela RNN.

*  Assuma um batch de frases de comprimento variável, como apresentado a seguir.

<img src="https://miro.medium.com/max/426/1*7qaPfwTbd6RBUWC5QNHxNw.png" width="350">

*  O empacotamento precisa receber os dados em ordem decrescente de comprimento, e internamente são criados "mini batches" com o seu batch. Dessa forma, apenas os timesteps que contém informação relevante sobre o dado são apresentadas à rede. Igualmente, somente esses timesteps impactam no backpropagation.

<img src="https://miro.medium.com/max/668/1*WF93EuCOGU834ENSnnofZg.png" width="360">


Para isso basta realizar o padding das suas sequências, **preservando os comprimetos originais** em outra variável. Na prática, o forward recebe mais um parâmetro, aqui chamamos de **```lengths```**, referente ao comprimento de cada amostra dentro do batch **```X```**, ordenado de forma descrescente.

Tendo em mãos (1) o batch de sequências preenchidas e ordenadas, e (2) o comprimento original de cada amostra, basta realizar as seguintes operações no forward da rede:

```python
## Empacote a sequência antes de alimentar a unidade recorrente
packed_input = nn.utils.rnn.pack_padded_sequence(X, lengths)

## Forward recorrente
packed_output, (hn, cn) = self.bilstm(packed_input, (h0, c0) )

## Desempacote a sequência para continuar o fluxo na rede.
output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
```

In [0]:
class BiLSTM(nn.Module):
  
  def __init__(self,input_size, hidden_size, num_layers, batch_size):
    super(BiLSTM, self).__init__()
    
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    
    self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True)
    
        
  def forward(self, X, lengths):
    
    h0 = torch.zeros(self.num_layers * 2, X.size(1), self.hidden_size).to(args['device'])
    c0 = torch.zeros(self.num_layers * 2, X.size(1), self.hidden_size).to(args['device'])
    
    ## Empacote a sequência antes de alimentar a unidade recorrente
    print('Input size:', X.size())
    packed_input = nn.utils.rnn.pack_padded_sequence(X, lengths)
    print('Packed size:', packed_input.data.size())
    
    ## Forward recorrente
    packed_output, (hn, cn) = self.bilstm(packed_input, (h0, c0) )
    print('Packed output size:', packed_output.data.size())

    ## Desempacote a sequência para continuar o fluxo na rede.
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    print('Unpacked output size:', output.size())
    
    return output
    

hidden_size   = 64
num_layers    = 1
input_size    = 1 
seq_len       = 140

net = BiLSTM(input_size,
             hidden_size, num_layers,
             args['batch_size']).to(args['device'])

# Generate random batch
seq_lens = [24, 21, 12, 11]
max_len  = max(seq_lens)

batch = [] 
for s in seq_lens:
  X = torch.randn(s, input_size)
  X = torch.cat( (X, torch.zeros(max_len - s, input_size)), dim=0 ).to(args['device'])
  
  batch.append(X)

batch = torch.stack(batch).permute((1,0,2))
output = net(batch, seq_lens)

Input size: torch.Size([24, 4, 1])
Packed size: torch.Size([68, 1])
Packed output size: torch.Size([68, 128])
Unpacked output size: torch.Size([24, 4, 128])


# Torchtext

Similar ao torchvision, o pacote torchtext facilita o trabalho com texto, oferecendo ferramentas aproveitáveis para outros contextos (séries temporais, dados tabulares etc.).

Vamos experimentar com o dado tabular a seguir, referente ao dataset Sentiment140 de análise de sentimentos, disponível em: http://help.sentiment140.com/for-students

In [0]:
!wget https://www.dropbox.com/s/u7jea1hynnubjb3/140_train_small.csv?dl=1
!mv '140_train_small.csv?dl=1' 140_train.csv

--2019-07-31 18:31:55--  https://www.dropbox.com/s/u7jea1hynnubjb3/140_train_small.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/u7jea1hynnubjb3/140_train_small.csv [following]
--2019-07-31 18:31:55--  https://www.dropbox.com/s/dl/u7jea1hynnubjb3/140_train_small.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2a4fa5db494a708bd8a9782fba.dl.dropboxusercontent.com/cd/0/get/AlxLOO3FWGbYi1akbIQsqKfH6Cxm8THpxdWCeaX1--KTTC6-nIs7xqlp8SjVT0_3MNSZA1uIYyJZVlA1V2BVW_DwBn_lcqcPcVAEHcET-3MZDc7lVzw8i3dhU1_Dn7wu4ek/file?dl=1# [following]
--2019-07-31 18:31:56--  https://uc2a4fa5db494a708bd8a9782fba.dl.dropboxusercontent.com/cd/0/get/AlxLOO3FWGbYi1akbIQsqKfH6Cxm8THpxdWCeaX1--KTTC6-nIs7xqlp8SjVT0_3MNSZA1uIYyJZVlA1V2BVW_Dw

## TabularDataset

Essa classe permite a construção de datasets acessando diretamente a tabela onde os dados estão contidos.
É possível filtrar as colunas que irão compor o dataset e ignorar o restante da informação através de ```Fields```, que nada mais são do que tipos de dados que incorporam instruções de transformações.

Documentação: https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.TabularDataset

In [0]:
import pandas as pd

df = pd.read_csv('140_train.csv')
df.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
106661,4,2193576427,Tue Jun 16 08:38:45 PDT 2009,NO_QUERY,wonder_nat,@AndrewDearling *yawns*
106662,4,2193577154,Tue Jun 16 08:38:49 PDT 2009,NO_QUERY,kcnitt,"oh yes, and btw, 8.00"
106663,4,2193577726,Tue Jun 16 08:38:52 PDT 2009,NO_QUERY,FrayBaby,@pokapolas love the donut and the toadstool.
106664,4,2193578347,Tue Jun 16 08:38:55 PDT 2009,NO_QUERY,CoachChic,"@BizCoachDeb Hey, I'm baack! And, thanks so m..."
106665,4,2193579249,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,razzberry5594,WOOOOO! Xbox is back


In [0]:
tokenize = lambda x: x.split()

TEXT = data.Field(tokenize=tokenize, include_lengths=True)
LABEL = data.LabelField(dtype=torch.float)

fields = [('label', LABEL), (None, None), (None, None), (None, None), (None, None), ('text', TEXT)]
train_data, test_data = data.TabularDataset.splits(
                                  path = '.',
                                  train = '140_train.csv',
                                  test = '140_train.csv',
                                  format = 'csv',
                                  fields = fields,
                                  skip_header = False)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

for sample in train_data:
  print(sample.text)
  print(sample.label)
  break
  
print(len(train_data), len(valid_data))

['@TheScottyDont', 'Awwww,', "you're", 'so', 'nice']
4
74667 32000


## Vocabulary

Uma pergunta que pode ter passado na sua cabeça: como alimentamos uma rede neural com palavras de um texto?

Para transformar palavras em dados numéricos, a solução mais simples é mapeá-las em um dicionário contendo o vocabulário completo do conjunto. 

<img src="https://static.packt-cdn.com/products/9781786465825/graphics/B05525_03_01.jpg" width="500">

Podemos fazer isso chamando a função **```build_vocab```** nos nossos fields. Como datasets de texto podem chegar a centenas de milhares de palavras, é importante definir um limite superior para o número de palavras mapeadas pelo dicionário. No código a seguir, esse limite é definido como ```MAX_VOCAB_SIZE = 25000```

Atenção também para o parâmetro ```vectors = "glove.6B.100d"```. O GloVe (Global Vectors) é um método de representação de palavras que explicaremos em maiores detalhes mais a frente. A princípio basta saber que o modelo "glove.**6B**.**100d**" foi treinado em **6 bilhões** de palavras e gera uma representação latente de dimensionalidade  **d = 100**

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

# Vocabulário com 25000 + <PAD> + <UNK>
len(TEXT.vocab)

.vector_cache/glove.6B.zip: 862MB [00:44, 19.2MB/s]                           
100%|█████████▉| 398734/400000 [00:22<00:00, 17162.31it/s]

25002

## BucketIterator
Essa classe funciona de forma análoga ao DataLoader que conhecemos,  porém leva em consideração a construção de batches contendo sequências de comprimento variável. Internamente ele agrega sequências de comprimento similar, minimizando a quantidade de padding necessária. Além disso, os dados já saem preparados para serem empacotados pela função ```pack_padded_sequence``` ordenados por comprimento de sequência e informando o comprimento real de cada amostra (sem padding).

Documentação: https://torchtext.readthedocs.io/en/latest/data.html?highlight=BucketIterator#torchtext.data.BucketIterator


In [0]:
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = args['batch_size'],
    sort_key = lambda x:len(x.text),
    sort_within_batch = True,
    device = args['device'])

In [0]:
for batch in train_iterator:
  text, lengths = batch.text
  print(text)
  print(lengths)
  print(batch.label)
  
  print([TEXT.vocab.itos[t] for t in text[0]])
  break

tensor([[ 3837,     0,   255,     0,     0,     0,     0,     0,     0,     0],
        [   22,  2161,     4,  9409,     8, 14343, 18952,   115, 23598,  6540],
        [  954,  1350,   817,    10,   158,     8,     0,    51,     4,    80],
        [   33,     7,    10,    67,   161,  6649,  1622,    55,    73,     0],
        [    9,    38,    40,     8,    50,  5337,   434, 11343,     0,    90],
        [   82,   402,    15,    57,  4386, 12675,     9,  6466, 10096,     2],
        [   10,    20, 10752,   631,   833,  6613,  3899,   178,     0,    50],
        [ 2459,  8950,     0,     1,     1,     1,     1,     1,     1,     1]],
       device='cuda:0')
tensor([8, 8, 8, 7, 7, 7, 7, 7, 7, 7], device='cuda:0')
tensor([0., 0., 1., 0., 1., 1., 1., 0., 0., 1.], device='cuda:0')
['@johncmayer', '<unk>', 'What', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


# Embedding Layer

Documentação Pytorch: https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding

Camadas de embedding são treinadas para mapear um ínidice numérico para um vetor denso de maior carga semântica.

Acabamos de ver a representação de palaras como ínidices de um vocabulário fixo. Apesar do índice informar a qual palavra estamos nos referindo, ele não incorpora nenhuma informação semântica sobre a palavra. O treinamento de embeddings para dados textuais tem como objetivo projetar esses índices em um espaço onde palavras semanticamente similares estejam próximas.

![](https://drive.google.com/uc?export=view&id=1pliMSOcjjOZAiR26ycowSeUJsj5cy9W_)

No Pytorch, a instância dessa classe recebe como parâmetro ```(vocab_size, embedding_size, padding_idx)```
* ```vocab_size```: Tamanho do vocabulário. Note que **não** se trata da dimensionalidade da entrada.
* ```embedding_size```: Dimensionalidade da dimensão latente. Caso haja o aproveitamento de embeddings pré treinadas deve-se definir a dimensionalidade da camada em função dos pesos que serão importados (ex: glove.6b.100d, ```embedding_size=100```).
* ```padding_idx```: Parâmetro **opcional** que retorna um tensor de zeros quando recebe instâncias com esse índice.

In [0]:
class BRNN(nn.Module):
  
  def __init__(self,vocab_size, embedding_size, hidden_size, num_layers, batch_size,
              embedding_weights=None, pad_idx=None, unk_idx=None):
    super(BRNN, self).__init__()
    
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    
    self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx=pad_idx)
    self.brnn  = nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=True)
    
    if embedding_weights is not None:
        self.init_embedding(embedding_weights, pad_idx, unk_idx, embedding_size)
        
  def init_embedding(self, weights, pad_idx, unk_idx, embedding_size):
    
    self.embed.weight.data.copy_(weights)
    self.embed.weight.data[unk_idx] = torch.zeros(embedding_size)
    self.embed.weight.data[pad_idx] = torch.zeros(embedding_size)
    
    
  def forward(self, X):
    pass
    

hidden_size    = 64
num_layers     = 1
embedding_size = 100
vocab_size     = len(TEXT.vocab)

pretrained_embeddings = TEXT.vocab.vectors
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

net = BRNN(vocab_size, embedding_size,
           hidden_size, num_layers,
           args['batch_size'],
           pretrained_embeddings, PAD_IDX,
           UNK_IDX).to(args['device'])

# Batch do Sentiment140
for batch in train_iterator:
  text, lengths = batch.text
  
  sample = text[:,0]
  
  print("Sequence length:", len(sample) )
  print("Vocabulary indices for each word:", sample)
  
  embedding = net.embed(sample)
  print("\n\nEmbedding output size:", embedding.size())
  print("Embedding vectors:", embedding)
  
  break
  

X = torch.randn(seq_len, args['batch_size'], input_size).to(args['device'])
output = net(X)

Sequence length: 27
Vocabulary indices for each word: tensor([ 5036,    96,   759,   468,     0,  3967,    27,    16,     0,    22,
           96,   168,    89,     0,     0, 24948,    96, 10568, 16894, 19965,
         2109,  4681,   137,    89,   339, 16416,  4238], device='cuda:0')


Embedding output size: torch.Size([27, 100])
Embedding vectors: tensor([[ 1.1866e+00, -5.6253e-01,  1.1038e+00,  ..., -3.4591e-01,
         -8.2398e-02,  2.3621e+00],
        [-1.7358e-01, -3.6788e-01, -2.9992e-01,  ..., -2.5977e-01,
          6.5464e-01,  4.6558e-01],
        [ 1.7581e+00,  2.2177e+00,  7.7441e-01,  ..., -2.3919e-01,
         -1.1639e+00,  5.1103e-01],
        ...,
        [-9.6620e-01, -3.3769e-01,  1.6463e-01,  ...,  4.0199e-01,
         -1.1374e+00,  1.9768e+00],
        [-2.4989e-01,  5.6004e-02,  1.2962e+00,  ..., -4.7342e-01,
         -2.1616e-01,  1.1519e-03],
        [ 5.7099e-01,  3.6458e-02,  1.0264e+00,  ...,  4.2581e-01,
          4.0433e-01,  3.3241e-01]], device='cuda:0',
